# Energy Resolution Calculation based on semi-empirical formula

In [10]:
import numpy as np
import pandas as pd

In [11]:
import matplotlib.pyplot as plt
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['legend.loc'] = 'upper right'
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['font.family'] = 'serif'

colors = ['#1f78b4', '#e66101', '#33a02c', '#984ea3', '#F27781', '#18298C', '#04BF8A', '#F2CF1D', '#F29F05', '#7155D9', 
          '#8D07F6', '#9E91F2', '#F29B9B', '#F25764', '#6FB7BF', '#B6ECF2', '#5D1314', '#B3640F']

$$ \frac{\sigma_{\langle E\rangle}}{{\langle E\rangle}} = \frac{\sqrt{F\cdot \langle E\rangle+  \frac{Q (1-e^{-t/\tau})}{e^{-t/\tau}} + \frac{Q^2}{(e^{-t/\tau})^2}\left(\frac{t}{\tau}\right)^2 \sigma_\tau^2 + \frac{\sigma^2_{Q,\mathrm{noise} }}{(e^{-t/\tau})^2}  + \frac{S}{\epsilon} \left[ (1-\epsilon) + \frac{\Lambda}{(1+\Lambda)^2} \right] + \frac{\mathrm{DC}+\eta_\mathrm{noise}}{\epsilon^2 (1+\Lambda)^2 } + S^2\sigma_{lm}^2 } }{{\langle E\rangle}} $$

## Constants

In [12]:
Fano = 1.0
CorrAv = 0.2 
ElecLifetime = 10.0 # in ms
PDE = 0.184
PTE = 0.332 
QValue = 2458000.0 # in eV
WValue = 11.5 # in eV
Quanta = QValue/WValue

## Functions

In [13]:
def GetLightNoise(PTE=0.3, PDE=0.2, CorrAv=0.2, LMapNoise=0.0, Yield=0.04, QValue=2458000): 
    Photons = QValue * Yield
    CollectionEff = PDE * PTE
    LightCollectionNoise = Photons / CollectionEff * (1 - CollectionEff)
    CorrelatedAvalancheNoise = Photons / CollectionEff * CorrAv / (1 + CorrAv)**2
    LightmapNoise = (Photons * LMapNoise)**2
    LightNoise = LightCollectionNoise + CorrelatedAvalancheNoise + LightmapNoise
    return LightNoise, LightCollectionNoise, CorrelatedAvalancheNoise, LightmapNoise

In [14]:
def GetChargeNoise(CurrentNoise=100, ElecLifetime=10.0, ElecLifetimeError=0.0, Yield=0.05, QValue=2458000): 
    Electrons = QValue * Yield
    DriftTime = 65.0 / 0.2 / 1000.0 # cm / cm/us = us
    ChargeNoiseLib = pd.read_csv('eventnoise_vs_currentnoise.csv', usecols=[0,1], header=None, names=['Current', 'Noise'])
    ElectronicsNoise = ChargeNoiseLib['Noise'].loc[ChargeNoiseLib['Current'] == CurrentNoise]
    ElectronicsNoise = ElectronicsNoise.to_numpy()[0]**2 / np.exp(-DriftTime/ElecLifetime)**2
    LifetimeNoise = Electrons * (1 - np.exp(-DriftTime/ElecLifetime)) / np.exp(-DriftTime/ElecLifetime)
    LifetimeSystematic = (Electrons / np.exp(-DriftTime/ElecLifetime) * DriftTime / ElecLifetime * ElecLifetimeError)**2
    ChargeNoise = ElectronicsNoise + LifetimeNoise + LifetimeSystematic
    return ChargeNoise, ElectronicsNoise, LifetimeNoise, LifetimeSystematic

In [15]:
def GetResolution(Noise, QValue=2458000.0, WValue=11.5):
    Quanta = QValue / WValue
    Resolution = np.sqrt(Noise) / Quanta * 100
    return Resolution

In [16]:
fig = plt.figure(figsize=(12,6))
plt.xticks(np.arange(0,40,1))
plt.xlabel('Noise Scenario')
plt.ylabel('Energy Resolutoin [%]')
plt.xlim(0,33)
plt.ylim(0,1.5)

NoiseTerms = {}
NoiseTerms['Light'],_,_,_ = GetLightNoise(PTE=0.337, PDE=0.283, CorrAv=CorrAv)
NoiseTerms['Charge'],_,_,_ = GetChargeNoise(CurrentNoise=100)
plt.scatter(1, GetResolution(NoiseTerms['Light']), marker='^', color='k')
plt.scatter(1, GetResolution(NoiseTerms['Charge']), marker='P', color='k')
plt.scatter(1, GetResolution(NoiseTerms['Charge'] + NoiseTerms['Light'] + Fano * Quanta), marker='o', color='k')

NoiseTerms['Light'],_,_,_ = GetLightNoise(PTE=0.332, PDE=0.184, CorrAv=CorrAv)
Offset = 2
for ii in range(6):
    X = ii + Offset
    CurrentNoise = 100 + ii*20
    NoiseTerms['Charge'],_,_,_ = GetChargeNoise(CurrentNoise=CurrentNoise)
    plt.scatter(X, GetResolution(NoiseTerms['Light']), marker='^', color='r')
    plt.scatter(X, GetResolution(NoiseTerms['Charge']), marker='P', color='r')
    plt.scatter(X, GetResolution(NoiseTerms['Charge'] + NoiseTerms['Light'] + Fano * Quanta), marker='o', color='r')

NoiseTerms['Charge'],_,_,_ = GetChargeNoise(CurrentNoise=100)
Offset = 8
for ii in range(6):
    X = ii + Offset
    LMapNoise = 0.001 + ii*0.002
    NoiseTerms['Light'],_,_,_ = GetLightNoise(PTE=0.332, PDE=0.184, CorrAv=CorrAv, LMapNoise=LMapNoise)
    plt.scatter(X, GetResolution(NoiseTerms['Light']), marker='^', color='b')
    plt.scatter(X, GetResolution(NoiseTerms['Charge']), marker='P', color='b')
    plt.scatter(X, GetResolution(NoiseTerms['Charge'] + NoiseTerms['Light'] + Fano * Quanta), marker='o', color='b')

NoiseTerms['Light'],_,_,_ = GetLightNoise(PTE=0.332, PDE=0.184, CorrAv=CorrAv)
Offset = 14
for ii in range(9):
    X = ii + Offset
    Lifetime = 10.0 - (ii+1)*1.0
    NoiseTerms['Charge'],_,_,_ = GetChargeNoise(CurrentNoise=100, ElecLifetime=Lifetime )
    plt.scatter(X, GetResolution(NoiseTerms['Light']), marker='^', color='orange')
    plt.scatter(X, GetResolution(NoiseTerms['Charge']), marker='P', color='orange')
    plt.scatter(X, GetResolution(NoiseTerms['Charge'] + NoiseTerms['Light'] + Fano * Quanta), marker='o', color='orange')

NoiseTerms['Light'],_,_,_ = GetLightNoise(PTE=0.332, PDE=0.184, CorrAv=CorrAv)
Offset = 23
for ii in range(10):
    X = ii + Offset
    LifetimeErr = 0.02 * (ii)
    NoiseTerms['Charge'],_,l_err,tau_err = GetChargeNoise(CurrentNoise=100, ElecLifetime=10.0, ElecLifetimeError=LifetimeErr)
    plt.scatter(X, GetResolution(NoiseTerms['Light']), marker='^', color='green')
    plt.scatter(X, GetResolution(NoiseTerms['Charge']), marker='P', color='green')
    plt.scatter(X, GetResolution(NoiseTerms['Charge'] + NoiseTerms['Light'] + Fano * Quanta), marker='o', color='green')

plt.legend(labels=['Light Resolution', 'Charge Resolution', 'Total Resolution'])
plt.savefig('res_vs_noise.png', dpi=300, bbox_inches='tight')

In [17]:
NoiseTerms['Light'],_,_,_ = GetLightNoise(PTE=0.337, PDE=0.184, CorrAv=0.2, LMapNoise=0.0)
NoiseTerms['Charge'],_,_,_ = GetChargeNoise(CurrentNoise=100, ElecLifetime=10.0, ElecLifetimeError=0.0)
total = GetResolution(NoiseTerms['Charge'] + NoiseTerms['Light'] + Fano * Quanta)
print('Baseline: %.2f' % total)

NoiseTerms['Light'],_,_,_ = GetLightNoise(PTE=0.37, PDE=0.28, CorrAv=0.15, LMapNoise=0.005)
NoiseTerms['Charge'],_,_,_ = GetChargeNoise(CurrentNoise=100, ElecLifetime=20.0, ElecLifetimeError=0.01)
total = GetResolution(NoiseTerms['Charge'] + NoiseTerms['Light'] + Fano * Quanta)
print('Baseline: %.2f' % total)

NoiseTerms['Light'],_,_,_ = GetLightNoise(PTE=0.20, PDE=0.13, CorrAv=0.4, LMapNoise=0.015)
NoiseTerms['Charge'],_,_,_ = GetChargeNoise(CurrentNoise=200, ElecLifetime=3.0, ElecLifetimeError=0.1)
total = GetResolution(NoiseTerms['Charge'] + NoiseTerms['Light'] + Fano * Quanta)
print('Baseline: %.2f' % total)

Baseline: 0.85
Baseline: 0.77
Baseline: 1.78


In [18]:
fig = plt.figure(figsize=(9,6))
plt.xlabel('Channel Energy Resolution [%]')
plt.ylabel('Rotated Energy Resolutoin [%]')
plt.xlim(0.4,1.2)
plt.ylim(0.6,1.3)

scale_pte = np.arange(0,0.23,0.01)
LightNoise = []
ChargeNoise,_,_,_ = GetChargeNoise(CurrentNoise=100)
ChargeNoise = [ChargeNoise]*len(scale_pte)
for x in scale_pte: 
  Light,_,_,_ = GetLightNoise(PTE=PTE-x, PDE=PDE, CorrAv=CorrAv)
  LightNoise.append(Light)

plt.plot(GetResolution(np.array(LightNoise)), GetResolution(np.array(ChargeNoise) + np.array(LightNoise) + Fano * Quanta), color='k', markerfacecolor='k', ls='-', label='Light Channel')

plt.plot(GetResolution(LightNoise[0]), GetResolution(np.array(ChargeNoise)[0] + np.array(LightNoise)[0] + Fano * Quanta), marker='o', color='k', markerfacecolor='k')
plt.plot(GetResolution(LightNoise[10]), GetResolution(np.array(ChargeNoise)[0] + np.array(LightNoise)[10] + Fano * Quanta), marker='o',color='k', markerfacecolor='k')
plt.plot(GetResolution(LightNoise[20]), GetResolution(np.array(ChargeNoise)[0] + np.array(LightNoise)[20] + Fano * Quanta), marker='o',  color='k', markerfacecolor='k')
plt.text(GetResolution(LightNoise[0]) +0.03, GetResolution(np.array(ChargeNoise)[0] + np.array(LightNoise)[0] + Fano * Quanta) -0.03, 'PTE={:0.3}%'.format((PTE-scale_pte[0])*100), fontsize=14)
plt.text(GetResolution(LightNoise[10])+0.03, GetResolution(np.array(ChargeNoise)[0] + np.array(LightNoise)[10] + Fano * Quanta)-0.03, 'PTE={:0.3}%'.format((PTE-scale_pte[10])*100), fontsize=14)
plt.text(GetResolution(LightNoise[20])+0.03, GetResolution(np.array(ChargeNoise)[0] + np.array(LightNoise)[20] + Fano * Quanta)-0.03, 'PTE={:0.3}%'.format((PTE-scale_pte[20])*100), fontsize=14)

scale_qnoise = np.arange(100,201,1)
ChargeNoise = []
LightNoise,_,_,_ = GetLightNoise(PTE=PTE, PDE=PDE, CorrAv=CorrAv)
LightNoise = [LightNoise]*len(scale_qnoise)
for x in scale_qnoise: 
  Charge,_,_,_ = GetChargeNoise(CurrentNoise=x)
  ChargeNoise.append(Charge)

plt.plot(GetResolution(np.array(ChargeNoise)), GetResolution(np.array(ChargeNoise) + np.array(LightNoise) + Fano * Quanta), color='k', markerfacecolor='k', ls='--', label='Charge Channel')

plt.plot(GetResolution(ChargeNoise[0]), GetResolution(np.array(ChargeNoise)[0] + np.array(LightNoise)[0] + Fano * Quanta), marker='o', color='k', markerfacecolor='w')
plt.plot(GetResolution(ChargeNoise[50]), GetResolution(np.array(ChargeNoise)[50] + np.array(LightNoise)[0] + Fano * Quanta), marker='o',color='k', markerfacecolor='w')
plt.plot(GetResolution(ChargeNoise[-1]), GetResolution(np.array(ChargeNoise)[-1] + np.array(LightNoise)[0] + Fano * Quanta), marker='o',  color='k', markerfacecolor='w')
plt.text(GetResolution(ChargeNoise[0]) -0.08, GetResolution(np.array(ChargeNoise)[0] + np.array(LightNoise)[0] + Fano * Quanta)+0.02, '100 e$^-$', fontsize=14)
plt.text(GetResolution(ChargeNoise[50])-0.08, GetResolution(np.array(ChargeNoise)[50] + np.array(LightNoise)[0] + Fano * Quanta)+0.02, '150 e$^-$', fontsize=14)
plt.text(GetResolution(ChargeNoise[-1])-0.08, GetResolution(np.array(ChargeNoise)[-1] + np.array(LightNoise)[0] + Fano * Quanta)+0.02, '200 e$^-$', fontsize=14)


plt.legend(loc='upper left')
plt.savefig('res_vs_channel)noise.png', dpi=600, bbox_inches='tight')